# Conversión de prescripciones en protocolos clínicos

In [1]:
import aclinprot as acp
import pandas as pd

## Procedimiento
- Exportar la prescripción en ARIA en formato `csv`
- Crear una primera version del protoccolo clínico a partir de la prescripción
- Exportar el archivo de estructuras del paciente
- Cambiar los nombres de las estructuras en la prescripción para hacerlos coincidir con los de contorneo

Crear protocolo clínico a partir de la prescripción

In [12]:
# Nombre del archivo csv con los datos de la prescripción
prescriptionFile = '../../prescripciones/SBRTPulmon5x10.csv'
# Campos del protocolo
ProtocolID='SBRTPulmon10Gyx5fx'
TreatmentSite='Lung'
PlanID='SBRTPulmon'

# Primera versión del protocolo clínico a partir de la prescripción
acp.convertPrescriptionIntoClinicalProtocol(prescriptionFile, ProtocolID, TreatmentSite, PlanID)

Nombres de las esructuras dados en el contorneo y en la prescripción

In [7]:
# Nombre del archivo de estructuras
structureFile = '../../DICOM/RS.1.2.246.352.71.4.964456977994.220341.20240109180651.dcm'

# DataFrame para comparar los nombres de las estructuras
contStrdf = pd.DataFrame(acp.readContouringStructureNames(structureFile), columns=['Contouring'])
presStrdf = pd.DataFrame(acp.readClinProtStructureNames('ClinicalProtocol.xml'), columns=['Prescription'])
pd.concat([contStrdf, presStrdf], axis=1)

,Contouring,Prescription
0,BODY,PTV SBRT
1,Ambos pulmones,suma pulmones-PTV
2,Corazón,medula
3,Esófago,piel
4,GTVt,pared torácica
5,Hígado,esófago
6,ITVt,via aerea
7,Médula,corazon
8,MTV,traquea
9,Pared costal,NaN


In [4]:
import pandas as pd

In [11]:
corsdf = pd.DataFrame([{'Old': 'suma pulmones-PTV', 'New' : 'Pulmones-GTVt'},
                       {'Old': 'medula', 'New' : 'Médula'},
                       {'Old': 'piel', 'New' : 'Piel'},
                       {'Old': 'pared torácica', 'New' : 'Pared costal'},
                       {'Old': 'esófago', 'New' : 'Esófago'},
                       {'Old': 'suma pulmones-PTV', 'New' : 'Pulmones-GTVt'},
                       {'Old': 'corazon', 'New' : 'Corazón'},
                       {'Old': 'traquea', 'New' : 'Tráquea'},
                       {'Old': 'PTV SBRT', 'New' : 'PTVt'}], index=range(9))
acp.correctStrNames(prescriptionFile, corsdf)

In [15]:
strNameSuggestions = acp.suggestStrNames(clinprot='ClinicalProtocol.xml',
                                         rsdicom=structureFile)
strNameSuggestions

,Structure,Suggestion
0,PTVt,PTVt
1,Pulmones-GTVt,Pulmones-GTVt
2,Médula,Médula
3,Piel,Piel
4,Pared costal,Pared costal
5,Esófago,Esófago
6,via aerea,via aerea
7,Corazón,Corazón
8,Tráquea,Tráquea


Corregir los nombres de las estructuras en la prescripción y recrear el protocolo clínico

In [14]:
# Corregir prescripción
strNamesChanges = strNameSuggestions.copy(deep=True)
strNamesChanges.rename({'Structure' : 'Old', 'Suggestion' : 'New'}, axis=1, inplace=True)
acp.correctStrNames(prescriptionFile, strNamesChanges)

# Recrear protocolo
acp.convertPrescriptionIntoClinicalProtocol(prescriptionFile, ProtocolID, TreatmentSite, PlanID)